In [1]:
import mne
import pandas as pd
import os
path = r'Aanand_19_01.csv'
sfreq = 120
#ch_names = ['Fp2', 'F4', 'F8', 'T8', 'Fp1', 'F3', 'F7', 'T7', 'Cz',
#            'C4', 'P4', 'O2', 'Pz', 'C3', 'P3', 'O1','stim']
df = pd.read_csv(path)
df.head()

,timestamps,Fp1,Fp2,C3,C4,P7,P8,O1,O2,F7,F8,F3,F4,T7,T8,P3,P4,stim,sfreq
0,1.709562e+09,-37315.566816,-37971.501109,-44301.649249,-39326.821485,-38091.932308,-42685.081683,-39995.362162,-45340.603035,-46383.580134,-44207.906033,-45612.847282,-82818.845847,-47694.107615,-55704.123462,-46313.172139,-56584.022234,0.0,125
1,1.709562e+09,-74676.597080,-75963.789340,-88632.936911,-78996.697545,-76203.310633,-85438.559704,-80021.703842,-90723.853198,-94142.060774,-87977.807281,-89926.566473,-114472.134646,-94306.346095,-180493.242203,-93103.375209,-106202.749157,0.0,125
2,1.709562e+09,-70195.832276,-71380.295918,-84370.906278,-75097.167503,-71921.431651,-81097.135675,-75507.679642,-86529.682461,-93178.342960,-84785.575841,-86176.167271,-113766.713591,-90730.156389,-180578.938792,-89466.031130,-104809.877850,0.0,125
3,1.709562e+09,-75372.898623,-76687.985860,-89306.842006,-79372.072741,-76887.944566,-86097.712648,-80749.387234,-91372.679636,-93151.386756,-85667.352160,-87404.350925,-113962.917204,-91830.532769,-180690.921031,-90602.080894,-105176.401755,0.0,125
4,1.709562e+09,-71427.502802,-72627.389148,-85527.877274,-76314.622320,-73081.621299,-82298.899567,-76746.994465,-87678.472719,-94171.565076,-87515.394391,-89263.390215,-114363.102837,-93716.796484,-180443.084889,-92489.417492,-106010.837079,0.0,125


In [2]:
temp = df.drop('stim', axis=1)

In [3]:
df = temp.drop('sfreq', axis=1)

In [5]:
eeg_data = df.iloc[:,1:18].values.T
eeg_data[:-1] = eeg_data[:-1] * (10*(-6))
eeg_data[:-1] = eeg_data[:-1] / 4

ch_names = ['Fp1',	'Fp2'	,'C3'	,'C4'	,'P7'	,'P8'	,'O1'	,'O2'	,'F7'	,'F8'	,'F3'	,'F4'	,'T7'	,'T8'	,'P3'	,'P4']
info = mne.create_info(ch_names = ch_names, sfreq = sfreq, ch_types = ( ["eeg" ]*16))
raw = mne.io.RawArray(eeg_data, info)

raw.set_eeg_reference()
#raw.filter(l_freq=1,h_freq=28)

Creating RawArray with float64 data, n_channels=16, n_times=65257
    Range : 0 ... 65256 =      0.000 ...   543.800 secs
Ready.


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,16 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,120.00 Hz
Highpass,0.00 Hz
Lowpass,60.00 Hz


In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import mne

# Define your duration values
durations = [2]

for duration in durations:
    # Assuming you have 'raw' defined somewhere
    epochs = mne.make_fixed_length_epochs(raw, duration=duration, overlap=0)
    
    eeg_data = epochs.get_data()
    
    condition_ranges = {'lev1': (50, 100),
                        'lev2': (140, 190),
                        'lev3': (210, 260)}
    
    epochs_by_condition = {}
    selected_epochs = []
    
    for condition, (start_epoch, end_epoch) in condition_ranges.items():
        condition_epochs = epochs[start_epoch:end_epoch + 1].copy()
        epochs_by_condition[condition] = condition_epochs
        selected_epochs.append(condition_epochs)
    
    selected_epochs = mne.concatenate_epochs(selected_epochs)
    
    for condition, epoch_data in epochs_by_condition.items():
        num_epochs = len(epoch_data)
        print(f"Number of epochs for {condition}: {num_epochs}")
    
    print(f"Total number of selected epochs: {len(selected_epochs)}")
    
    X = selected_epochs.get_data(copy=True)
    y = np.concatenate([np.full(len(epochs), i) for i, epochs in enumerate(epochs_by_condition.values())])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Logistic Regression
    model = LogisticRegression(max_iter=5000)
    model.fit(X_train.reshape(len(X_train), -1), y_train)
    y_pred = model.predict(X_test.reshape(len(X_test), -1))
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy of Logistic Regression for duration {duration}: {accuracy}")
    
    # Random Forest
    rf_model = RandomForestClassifier(n_estimators=175, random_state=42)
    rf_model.fit(X_train.reshape(len(X_train), -1), y_train)
    y_pred_rf = rf_model.predict(X_test.reshape(len(X_test), -1))
    accuracy_rf = accuracy_score(y_test, y_pred_rf)
    print(f"Random Forest Accuracy for duration {duration}: {accuracy_rf}")
    
    # Gradient Boosting
    gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
    gb_model.fit(X_train.reshape(len(X_train), -1), y_train)
    y_pred_gb = gb_model.predict(X_test.reshape(len(X_test), -1))
    accuracy_gb = accuracy_score(y_test, y_pred_gb)
    print(f"Gradient Boosting Accuracy for duration {duration}: {accuracy_gb}")


Not setting metadata
271 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 271 events and 240 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 51 events and 240 original time points ...


Using data from preloaded Raw for 51 events and 240 original time points ...
Using data from preloaded Raw for 51 events and 240 original time points ...
Not setting metadata
153 matching events found
No baseline correction applied
Number of epochs for lev1: 51
Number of epochs for lev2: 51
Number of epochs for lev3: 51
Total number of selected epochs: 153
Accuracy of Logistic Regression for duration 2: 1.0
Random Forest Accuracy for duration 2: 1.0
Gradient Boosting Accuracy for duration 2: 1.0
